In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('employee_burnout.csv')

# Handle missing values
data.fillna(data.median(numeric_only=True), inplace=True)
data.fillna(data.mode().iloc[0], inplace=True)

# Convert Date of Joining to a numerical feature (e.g., days since joining)
data['Date of Joining'] = pd.to_datetime(data['Date of Joining'], format='%d-%m-%Y', errors='coerce')
data['Days Since Joining'] = (pd.Timestamp.now() - data['Date of Joining']).dt.days

# Drop the original Date of Joining column
data.drop(columns=['Date of Joining'], inplace=True)

# Encode categorical variables using Label Encoding
label_encoders = {}
for column in ['Gender', 'Company Type', 'WFH Setup Available']:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Define features and target
X = data.drop(columns=['Employee ID', 'Burn Rate'])
y = data['Burn Rate']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
accuracy = model.score(X_test, y_test) * 100

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")
print(f"Accuracy: {accuracy:.2f}%")

# Feature importance
importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df.sort_values(by='Importance', ascending=False, inplace=True)

print("\nFeature Importances:")
print(importance_df)


Mean Squared Error: 0.01
R-squared: 0.84
Accuracy: 83.60%

Feature Importances:
                Feature  Importance
5  Mental Fatigue Score    0.784615
4   Resource Allocation    0.092618
6    Days Since Joining    0.081785
3           Designation    0.016360
1          Company Type    0.008462
0                Gender    0.008096
2   WFH Setup Available    0.008064
